In [11]:
!pip install -q pandas scikit-learn textstat joblib

In [12]:

import pandas as pd
import re
import textstat
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [13]:

df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None)
df = df[[0, 5]]
df.columns = ['label', 'text']
df = df.sample(5000).reset_index(drop=True)

def advanced_professionalism_score(text):
    text = text.lower()
    score = 0
    if re.search(r"\b(yo|dude|bro|u|lmao|lol|wtf|gonna|wanna|thx|btw)\b", text):
        score -= 2
    if re.search(r"(regards|sincerely|kindly|thank you|attached|please find|i hope this email|looking forward to working)", text):
        score += 3  # Stronger reward for formal phrases
    if "'" in text or re.search(r"\b(can't|won't|i'm|it's|don't)\b", text):
        score -= 1
    try:
        grade = textstat.flesch_kincaid_grade(text)
        if grade > 10: score += 1
        elif grade < 6: score -= 1
        avg_len = textstat.avg_sentence_length(text)
        if avg_len > 15: score += 1
        elif avg_len < 8: score -= 1
    except:
        pass
    return 0 if score <= -1 else 1 if score <= 1 else 2

df['label'] = df['text'].apply(advanced_professionalism_score)

def clean(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[^a-zA-Z ]", "", text)
    return text.lower()

df['text_clean'] = df['text'].apply(clean)


C:\Users\bazej\AppData\Local\Temp\ipykernel_11492\2843796476.py:19: DeprecationWarning: The 'avg_sentence_length' method has been deprecated due to being the same as 'words_per_sentence'. This method will be removed in thefuture.
  avg_len = textstat.avg_sentence_length(text)


In [14]:

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['text_clean'])
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.82      1.00      0.90       409
           1       0.00      0.00      0.00        83
           2       0.00      0.00      0.00         8

    accuracy                           0.82       500
   macro avg       0.27      0.33      0.30       500
weighted avg       0.67      0.82      0.74       500



c:\Users\bazej\Desktop\email_professionalism_app\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\bazej\Desktop\email_professionalism_app\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\bazej\Desktop\email_professionalism_app\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag

In [15]:

joblib.dump(clf, "model.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")


['vectorizer.pkl']